In [137]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [138]:
# Ler o arquivo com os dados, mostre uma amostra do arquivo e exibe a 
# contagem de cada uma das colunas.

In [139]:
dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')

In [140]:
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [142]:
# Vamos contar quantos tweets de cada tipo existem: neutro, positivo e negativo.

In [143]:
dataset[dataset.Classificacao=='Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [144]:
dataset[dataset.Classificacao=='Positivo'].count()

Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [145]:
dataset[dataset.Classificacao=='Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [146]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ellen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [147]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ellen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [148]:
# definimos as stopwords em portugues

In [149]:
stopwords_list = stopwords.words('portuguese')

In [150]:
print(stopwords_list)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [151]:
# Precisamos criar váriaveis diferentes para armazenar os tweets e a sua classificação

In [152]:
tweets = dataset['Text'].values
classes = dataset['Classificacao'].values

In [153]:
# variaveis em lowercase

In [154]:
tweets_lower = []
for tweet in tweets:
    tweets_lower.append(tweet.lower())

In [155]:
# remover stopwords

In [165]:
tweets_sem_stopwords = []
for tweet in tweets_lower:
    new_tweet = ""
    text = tweet.split(" ")
    for word in text: 
        if word in stopwords_list:
            text.remove(word)
        else:
            new_tweet = new_tweet + " " + word
    tweets_sem_stopwords.append(new_tweet)

In [ ]:
# teste para prever resultado

In [131]:
testes = ['O governo de Minas é uma tragédia, muito ruim','Estou muito feliz com o governo de Minas esse ano','O estado de Minas Gerais decretou calamidade financeira!!!','A segurança do estado está deixando a desejar','O governador de Minas é do Novo']
print(testes)

['O governo de Minas é uma tragédia, muito ruim', 'Estou muito feliz com o governo de Minas esse ano', 'O estado de Minas Gerais decretou calamidade financeira!!!', 'A segurança do estado está deixando a desejar', 'O governador de Minas é do Novo']


In [132]:
# Calculo a BoW dos tweets dentro da variável testes usando o TF.

In [167]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets_sem_stopwords)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [168]:
freq_testes = vectorizer.transform(testes)

In [169]:
print(freq_testes)

  (0, 26342)	0.6273567354969901
  (0, 20978)	0.6000599974090601
  (0, 20498)	0.2690008465823315
  (0, 16221)	0.19608807615684895
  (0, 12032)	0.36815694596611015
  (1, 20978)	0.4596307390191682
  (1, 20498)	0.20604782595953144
  (1, 16221)	0.15019849306065172
  (1, 14879)	0.40905234152287584
  (1, 14348)	0.4805393480821634
  (1, 12032)	0.28199888324515104
  (1, 11076)	0.3392025192934948
  (1, 2557)	0.3618020003278162
  (2, 20531)	0.25117014372707697
  (2, 20498)	0.2158020984633054
  (2, 15741)	0.15771174741890534
  (2, 15074)	0.16918233863523935
  (2, 14074)	0.45380084207092397
  (2, 14024)	0.14539125676731057
  (2, 12094)	0.5032880071769694
  (2, 12093)	0.39819300459971385
  (2, 12032)	0.29534866716120844
  (2, 4441)	0.2537671895949035
  (2, 4439)	0.21514744623437024
  (3, 26761)	0.3901557079258399
  (3, 14378)	0.5443011942100329
  (3, 14024)	0.15723926173014913
  (3, 12827)	0.5443011942100329
  (3, 12826)	0.48013202153835605
  (4, 21657)	0.37100706304805786
  (4, 20498)	0.32625453119

In [170]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB()

In [171]:
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Neutro', 'Neutro'], dtype='<U8')

In [172]:
metrics.accuracy_score(classes,resultados)

0.871447737528967

In [173]:
print(metrics.classification_report(classes,resultados))

              precision    recall  f1-score   support

    Negativo       0.87      0.91      0.89      2446
      Neutro       0.84      0.78      0.81      2453
    Positivo       0.90      0.91      0.91      3300

    accuracy                           0.87      8199
   macro avg       0.87      0.87      0.87      8199
weighted avg       0.87      0.87      0.87      8199



In [174]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   Negativo  Neutro  Positivo   All
Real                                      
Negativo      2219     163        64  2446
Neutro         258    1910       285  2453
Positivo        85     199      3016  3300
All           2562    2272      3365  8199
